In [1]:
import pandas as pd
import pandera as pa
from pandera import Column, DataFrameSchema, Check, Index

In [2]:
#Leitura de base de dados: df: dataframe (sep=tipo de separador';') (Parse_dates: para entender dados como data) 
#(df.head(x): escolher quantos dados exibir) (dayfirst=true: para avisar que primeiro dado é referente ao dia e não o mês)

df = pd.read_csv("ocorrencia.csv", sep=";", parse_dates=['ocorrencia_dia'], dayfirst=True)
df.head(10)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
2,45333,45333,ACIDENTE,VIAMÃO,RS,****,2012-01-06,13:00:00,0
3,45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,****,2012-01-06,17:00:00,0
4,45407,45407,ACIDENTE,SÃO SEPÉ,RS,****,2012-01-06,16:30:00,0
5,52243,52243,INCIDENTE,UBATUBA,SP,****,2012-01-06,14:30:00,0
6,50713,50713,INCIDENTE GRAVE,CAMPINAS,SP,SDAI,2012-01-07,18:15:00,0
7,45334,45334,INCIDENTE,BELÉM,PA,SBBE,2012-01-08,19:12:00,0
8,45391,45391,ACIDENTE,CONCEIÇÃO DAS ALAGOAS,MG,****,2012-01-08,16:00:00,0
9,52244,52244,INCIDENTE,UBERLÂNDIA,MG,SBUL,2012-01-08,22:13:00,0


In [3]:
#criamos o schema para validar os tipos de dados: ocorrencia_hora: nullable (permite dados nulos)
schema = pa.DataFrameSchema(
    columns = {
        "codigo_ocorrencia":pa.Column(pa.Int),
        "codigo_ocorrencia2":pa.Column(pa.Int),
        "ocorrencia_classificacao":pa.Column(pa.String),
        "ocorrencia_cidade":pa.Column(pa.String),
        "ocorrencia_uf":pa.Column(pa.String, Check.str_length(2,2)), #limitamos o tamanho de carateres
        "ocorrencia_aerodromo":pa.Column(pa.String),
        "ocorrencia_dia":pa.Column(pa.DateTime),
        "ocorrencia_hora":pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'), nullable=True),
        "total_recomendacoes":pa.Column(pa.Int)
    }
)

In [4]:
#validação do schema montado acima:
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
2,45333,45333,ACIDENTE,VIAMÃO,RS,****,2012-01-06,13:00:00,0
3,45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,****,2012-01-06,17:00:00,0
4,45407,45407,ACIDENTE,SÃO SEPÉ,RS,****,2012-01-06,16:30:00,0
...,...,...,...,...,...,...,...,...,...
5162,80458,80458,ACIDENTE,JATAÍ,GO,####,2021-12-30,20:30:00,0
5163,80452,80452,ACIDENTE,MARACAÍ,SP,*****,2021-12-31,09:30:00,0
5164,80457,80457,INCIDENTE GRAVE,NOVO HAMBURGO,RS,SSNH,2021-12-31,11:59:00,0
5165,80460,80460,INCIDENTE,CURITIBA,PR,SBBI,2021-12-31,15:12:00,0


In [5]:
#conferindo se os tipos de dados já estão prontos...

df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object

In [6]:
df.dtypes
df.ocorrencia_dia.dt.month

0        1
1        1
2        1
3        1
4        1
        ..
5162    12
5163    12
5164    12
5165    12
5166    12
Name: ocorrencia_dia, Length: 5167, dtype: int64

# Limpeza

In [7]:
df.loc[1,"ocorrencia_cidade"] #consulda de dados por cabeçalho

'GUARULHOS'

In [8]:
df.loc[3] #consulta de uma linha inteira

codigo_ocorrencia                         45401
codigo_ocorrencia2                        45401
ocorrencia_classificacao               ACIDENTE
ocorrencia_cidade                 SÃO SEBASTIÃO
ocorrencia_uf                                SP
ocorrencia_aerodromo                       ****
ocorrencia_dia              2012-01-06 00:00:00
ocorrencia_hora                        17:00:00
total_recomendacoes                           0
Name: 3, dtype: object

In [9]:
df.loc[1:3] #consulta intervalo

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
2,45333,45333,ACIDENTE,VIAMÃO,RS,****,2012-01-06,13:00:00,0
3,45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,****,2012-01-06,17:00:00,0


In [10]:
df.loc[[10,40]] #linhas específicas, 10 e 40

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
10,45332,45332,ACIDENTE,VIAMÃO,RS,****,2012-01-09,13:30:00,0
40,45415,45415,ACIDENTE,RIO DE JANEIRO,RJ,****,2012-01-30,13:55:00,0


In [11]:
df.loc[:,"ocorrencia_cidade"] #filtrando somente a coluna "ocorrencia cidade"

0        PORTO ALEGRE
1           GUARULHOS
2              VIAMÃO
3       SÃO SEBASTIÃO
4            SÃO SEPÉ
            ...      
5162            JATAÍ
5163          MARACAÍ
5164    NOVO HAMBURGO
5165         CURITIBA
5166        PETROLINA
Name: ocorrencia_cidade, Length: 5167, dtype: object

In [12]:
df.codigo_ocorrencia.is_unique #verifica se tem dados repetidos no código de ocorrência

True

In [13]:
df.set_index('codigo_ocorrencia', inplace=True) #cria data frame com index na coluna 'codigo de ocorrencia
#inplace: modifica o dataframe in place(sem criar novo)

In [14]:
df.head()

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
codigo_ocorrencia,,,,,,,,
52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0
45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
45333,45333,ACIDENTE,VIAMÃO,RS,****,2012-01-06,13:00:00,0
45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,****,2012-01-06,17:00:00,0
45407,45407,ACIDENTE,SÃO SEPÉ,RS,****,2012-01-06,16:30:00,0


In [16]:
df.loc[45407] #a impresão (consulta) acontece pela nova coluna de inderx('codigo de ocorrencia')

codigo_ocorrencia2                        45407
ocorrencia_classificacao               ACIDENTE
ocorrencia_cidade                      SÃO SEPÉ
ocorrencia_uf                                RS
ocorrencia_aerodromo                       ****
ocorrencia_dia              2012-01-06 00:00:00
ocorrencia_hora                        16:30:00
total_recomendacoes                           0
Name: 45407, dtype: object

In [25]:
df.reset_index(drop=True, inplace=True) #igual ao anterior, só que Drop=true recria a coluna index para o dataframe

In [26]:
df.head()

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,52242.0,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0.0
1,45331.0,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3.0
2,45333.0,ACIDENTE,VIAMÃO,RS,****,2012-01-06,13:00:00,0.0
3,45401.0,ACIDENTE,SÃO SEBASTIÃO,SP,****,2012-01-06,17:00:00,0.0
4,45407.0,ACIDENTE,SÃO SEPÉ,RS,****,2012-01-06,16:30:00,0.0


In [27]:
df.loc[0,'ocorrencia_aerodromo'] = '' #trocamos o valor de essa celula por espaço vazio

In [28]:
df.head(1)

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,52242.0,INCIDENTE,PORTO ALEGRE,RS,,2012-01-05,20:27:00,0.0


In [29]:
df.loc[1] = 20 #em toda a linha [1] os valores foram trocados por 20

In [30]:
df.head(2)

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,52242.0,INCIDENTE,PORTO ALEGRE,RS,,2012-01-05 00:00:00,20:27:00,0.0
1,20.0,20,20,20,20,20,20,20.0


In [31]:
df.loc[:,'total_recomendacoes'] = 10 #coluna de recomendações modificado para 10, em todos

In [32]:
df

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,52242.0,INCIDENTE,PORTO ALEGRE,RS,,2012-01-05 00:00:00,20:27:00,10
1,20.0,20,20,20,20,20,20,10
2,45333.0,ACIDENTE,VIAMÃO,RS,****,2012-01-06 00:00:00,13:00:00,10
3,45401.0,ACIDENTE,SÃO SEBASTIÃO,SP,****,2012-01-06 00:00:00,17:00:00,10
4,45407.0,ACIDENTE,SÃO SEPÉ,RS,****,2012-01-06 00:00:00,16:30:00,10
...,...,...,...,...,...,...,...,...
5163,80452.0,ACIDENTE,MARACAÍ,SP,*****,2021-12-31 00:00:00,09:30:00,10
5164,80457.0,INCIDENTE GRAVE,NOVO HAMBURGO,RS,SSNH,2021-12-31 00:00:00,11:59:00,10
5165,80460.0,INCIDENTE,CURITIBA,PR,SBBI,2021-12-31 00:00:00,15:12:00,10
5166,80467.0,INCIDENTE,PETROLINA,PE,SBPL,2021-12-31 00:00:00,20:30:00,10


In [36]:
df.loc[df.ocorrencia_uf == 'SP', ['ocorrencia_classificacao']] = 'GRAVE' #todos os que são SP passa a grave

In [38]:
df.loc[df.ocorrencia_uf == 'SP']

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_uf_bkp
3,45401.0,GRAVE,SÃO SEBASTIÃO,SP,****,2012-01-06 00:00:00,17:00:00,10,SP
5,52243.0,GRAVE,UBATUBA,SP,****,2012-01-06 00:00:00,14:30:00,10,SP
6,50713.0,GRAVE,CAMPINAS,SP,SDAI,2012-01-07 00:00:00,18:15:00,10,SP
13,45408.0,GRAVE,ELDORADO,SP,****,2012-01-11 00:00:00,13:45:00,10,SP
26,45410.0,GRAVE,BRAGANÇA PAULISTA,SP,SDVH,2012-01-21 00:00:00,23:30:00,10,SP
...,...,...,...,...,...,...,...,...,...
5148,80441.0,GRAVE,BRAGANÇA PAULISTA,SP,SDVH,2021-12-20 00:00:00,18:15:00,10,SP
5157,80453.0,GRAVE,CAMPINAS,SP,SBKP,2021-12-29 00:00:00,09:00:00,10,SP
5158,80454.0,GRAVE,ARAÇATUBA,SP,SBAU,2021-12-29 00:00:00,21:35:00,10,SP
5161,80456.0,GRAVE,SÃO PAULO,SP,SBSP,2021-12-30 00:00:00,13:15:00,10,SP


In [39]:
df.loc[df.ocorrencia_aerodromo == '****', ['ocorrencia_aerodromo']] = pd.NA #aqui só trocamos o caso de '****' na coluna por NA

In [40]:
df.head()

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_uf_bkp
0,52242.0,INCIDENTE,PORTO ALEGRE,RS,,2012-01-05 00:00:00,20:27:00,10,RS
1,20.0,20,20,20,20,20,20,10,20
2,45333.0,ACIDENTE,VIAMÃO,RS,<NA>,2012-01-06 00:00:00,13:00:00,10,RS
3,45401.0,GRAVE,SÃO SEBASTIÃO,SP,<NA>,2012-01-06 00:00:00,17:00:00,10,SP
4,45407.0,ACIDENTE,SÃO SEPÉ,RS,<NA>,2012-01-06 00:00:00,16:30:00,10,RS


In [ ]:
df.replace(['**', '###!', '####','****','*****','NULL'], pd.NA, inplace=True) #aqui os casos faltantes, em todas as colunas por NA

In [41]:
df.head()

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_uf_bkp
0,52242.0,INCIDENTE,PORTO ALEGRE,RS,,2012-01-05 00:00:00,20:27:00,10,RS
1,20.0,20,20,20,20,20,20,10,20
2,45333.0,ACIDENTE,VIAMÃO,RS,<NA>,2012-01-06 00:00:00,13:00:00,10,RS
3,45401.0,GRAVE,SÃO SEBASTIÃO,SP,<NA>,2012-01-06 00:00:00,17:00:00,10,SP
4,45407.0,ACIDENTE,SÃO SEPÉ,RS,<NA>,2012-01-06 00:00:00,16:30:00,10,RS


In [ ]:
df.isna().sum() #contando os casos de NA em toda a tabela

# Transformação

In [96]:
#recarregamos a tabela, e criamos uma lista com valores ausentes
valores_ausentes = ['**', '###!', '####','****','*****','NULL'] 
df = pd.read_csv("ocorrencia.csv", sep=";", parse_dates=['ocorrencia_dia'], dayfirst=True, na_values=valores_ausentes)
df.head(10)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
2,45333,45333,ACIDENTE,VIAMÃO,RS,NaN,2012-01-06,13:00:00,0
3,45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,NaN,2012-01-06,17:00:00,0
4,45407,45407,ACIDENTE,SÃO SEPÉ,RS,NaN,2012-01-06,16:30:00,0
5,52243,52243,INCIDENTE,UBATUBA,SP,NaN,2012-01-06,14:30:00,0
6,50713,50713,INCIDENTE GRAVE,CAMPINAS,SP,SDAI,2012-01-07,18:15:00,0
7,45334,45334,INCIDENTE,BELÉM,PA,SBBE,2012-01-08,19:12:00,0
8,45391,45391,ACIDENTE,CONCEIÇÃO DAS ALAGOAS,MG,NaN,2012-01-08,16:00:00,0
9,52244,52244,INCIDENTE,UBERLÂNDIA,MG,SBUL,2012-01-08,22:13:00,0


In [98]:
#repetimos o schema

schema = pa.DataFrameSchema(
    columns = {
        "codigo_ocorrencia": pa.Column(pa.Int),
        "codigo_ocorrencia2": pa.Column(pa.Int),
        "ocorrencia_classificacao": pa.Column(pa.String),
        "ocorrencia_cidade": pa.Column(pa.String),
        "ocorrencia_uf": pa.Column(pa.String, pa.Check.str_length(2,2), nullable=True),
        "ocorrencia_aerodromo": pa.Column(pa.String, nullable=True),
        "ocorrencia_dia": pa.Column(pa.DateTime),
        "ocorrencia_hora": pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'), nullable=True),
        "total_recomendacoes": pa.Column(pa.Int)   
    }
)

In [99]:
schema.validate(df) #validar schema

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
2,45333,45333,ACIDENTE,VIAMÃO,RS,NaN,2012-01-06,13:00:00,0
3,45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,NaN,2012-01-06,17:00:00,0
4,45407,45407,ACIDENTE,SÃO SEPÉ,RS,NaN,2012-01-06,16:30:00,0
...,...,...,...,...,...,...,...,...,...
5162,80458,80458,ACIDENTE,JATAÍ,GO,NaN,2021-12-30,20:30:00,0
5163,80452,80452,ACIDENTE,MARACAÍ,SP,NaN,2021-12-31,09:30:00,0
5164,80457,80457,INCIDENTE GRAVE,NOVO HAMBURGO,RS,SSNH,2021-12-31,11:59:00,0
5165,80460,80460,INCIDENTE,CURITIBA,PR,SBBI,2021-12-31,15:12:00,0


In [100]:
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object

In [101]:
df.loc[1]

codigo_ocorrencia                         45331
codigo_ocorrencia2                        45331
ocorrencia_classificacao               ACIDENTE
ocorrencia_cidade                     GUARULHOS
ocorrencia_uf                                SP
ocorrencia_aerodromo                       SBGR
ocorrencia_dia              2012-01-06 00:00:00
ocorrencia_hora                        13:44:00
total_recomendacoes                           3
Name: 1, dtype: object

In [102]:
df.iloc[1] #consulta pelo comando iloc

codigo_ocorrencia                         45331
codigo_ocorrencia2                        45331
ocorrencia_classificacao               ACIDENTE
ocorrencia_cidade                     GUARULHOS
ocorrencia_uf                                SP
ocorrencia_aerodromo                       SBGR
ocorrencia_dia              2012-01-06 00:00:00
ocorrencia_hora                        13:44:00
total_recomendacoes                           3
Name: 1, dtype: object

In [44]:
df.iloc[-1]

codigo_ocorrencia                         80467
codigo_ocorrencia2                        80467
ocorrencia_classificacao              INCIDENTE
ocorrencia_cidade                     PETROLINA
ocorrencia_uf                                PE
ocorrencia_aerodromo                       SBPL
ocorrencia_dia              2021-12-31 00:00:00
ocorrencia_hora                        20:30:00
total_recomendacoes                           0
Name: 5166, dtype: object

In [103]:
df.tail() #retorna as últimas linhas

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
5162,80458,80458,ACIDENTE,JATAÍ,GO,NaN,2021-12-30,20:30:00,0
5163,80452,80452,ACIDENTE,MARACAÍ,SP,NaN,2021-12-31,09:30:00,0
5164,80457,80457,INCIDENTE GRAVE,NOVO HAMBURGO,RS,SSNH,2021-12-31,11:59:00,0
5165,80460,80460,INCIDENTE,CURITIBA,PR,SBBI,2021-12-31,15:12:00,0
5166,80467,80467,INCIDENTE,PETROLINA,PE,SBPL,2021-12-31,20:30:00,0


In [104]:
df.iloc[10:15] #iloc: consulta do indice 10, até o 14 (valor 15 é de corte), total: 5 consultas

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
10,45332,45332,ACIDENTE,VIAMÃO,RS,NaN,2012-01-09,13:30:00,0
11,52245,52245,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-09,13:36:00,0
12,45396,45396,INCIDENTE GRAVE,MARABÁ,PA,SBMA,2012-01-11,11:21:00,0
13,45408,45408,ACIDENTE,ELDORADO,SP,NaN,2012-01-11,13:45:00,1
14,45447,45447,INCIDENTE,RIO BRANCO,AC,NaN,2012-01-13,18:15:00,0


In [105]:
df.loc[10:15] #loc: consulta do indice 10 até o 15, total: 6 consultas

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
10,45332,45332,ACIDENTE,VIAMÃO,RS,NaN,2012-01-09,13:30:00,0
11,52245,52245,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-09,13:36:00,0
12,45396,45396,INCIDENTE GRAVE,MARABÁ,PA,SBMA,2012-01-11,11:21:00,0
13,45408,45408,ACIDENTE,ELDORADO,SP,NaN,2012-01-11,13:45:00,1
14,45447,45447,INCIDENTE,RIO BRANCO,AC,NaN,2012-01-13,18:15:00,0
15,45409,45409,ACIDENTE,CÁCERES,MT,NaN,2012-01-14,10:00:00,0


In [47]:
df.loc[:,'ocorrencia_uf'] #só a coluna para ver a UF

0       RS
1       SP
2       RS
3       SP
4       RS
        ..
5162    GO
5163    SP
5164    RS
5165    PR
5166    PE
Name: ocorrencia_uf, Length: 5167, dtype: object

In [48]:
df['ocorrencia_uf'] #mesmo retorno ao comando anterior

0       RS
1       SP
2       RS
3       SP
4       RS
        ..
5162    GO
5163    SP
5164    RS
5165    PR
5166    PE
Name: ocorrencia_uf, Length: 5167, dtype: object

In [49]:
df.isna().sum() #soma valores NA

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  2
ocorrencia_aerodromo        1905
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

In [50]:
df.isnull().sum() #mesmo retorno, celular com valor 'NA' tem mesmo que valor 'null'

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  2
ocorrencia_aerodromo        1905
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

In [51]:
filtro = df.ocorrencia_uf.isnull() #consulta de ocorrencia 'null'
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
1099,49474,49474,ACIDENTE,ÁGUAS INTERNACIONAIS,NaN,NaN,2013-09-02,02:54:00,0
4728,79956,79956,ACIDENTE,NÃO IDENTIFICADA,NaN,NaN,2021-02-26,11:00:00,0


In [52]:
filtro = df.ocorrencia_aerodromo.isnull() #ocorrencia aerodromo 'null'
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
2,45333,45333,ACIDENTE,VIAMÃO,RS,NaN,2012-01-06,13:00:00,0
3,45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,NaN,2012-01-06,17:00:00,0
4,45407,45407,ACIDENTE,SÃO SEPÉ,RS,NaN,2012-01-06,16:30:00,0
5,52243,52243,INCIDENTE,UBATUBA,SP,NaN,2012-01-06,14:30:00,0
8,45391,45391,ACIDENTE,CONCEIÇÃO DAS ALAGOAS,MG,NaN,2012-01-08,16:00:00,0
...,...,...,...,...,...,...,...,...,...
5151,80446,80446,ACIDENTE,JACAREACANGA,PA,NaN,2021-12-20,17:10:00,0
5155,80450,80450,ACIDENTE,DOURADOS,MS,NaN,2021-12-28,10:30:00,0
5159,80455,80455,INCIDENTE GRAVE,SANTA RITA,PB,NaN,2021-12-29,18:50:00,0
5162,80458,80458,ACIDENTE,JATAÍ,GO,NaN,2021-12-30,20:30:00,0


In [106]:
filtro = df.ocorrencia_hora.isnull() #hora como 'null'
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
2974,78823,78823,INCIDENTE,CORUMBÁ,MS,NaN,2017-07-09,NaN,0


In [107]:
df.count() #verificar se temos mesma quantidade de dados em cada coluna (ver em ocorrencia hora, e aerodromo por exemplo)

codigo_ocorrencia           5167
codigo_ocorrencia2          5167
ocorrencia_classificacao    5167
ocorrencia_cidade           5167
ocorrencia_uf               5165
ocorrencia_aerodromo        3262
ocorrencia_dia              5167
ocorrencia_hora             5166
total_recomendacoes         5167
dtype: int64

In [54]:
#ocorrencias com mais de 10 recomendações
filtro = df.total_recomendacoes > 10
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
844,47938,47938,INCIDENTE,BRASÍLIA,DF,SBBR,2013-04-13,18:00:00,11
1669,52265,52265,ACIDENTE,SANTOS,SP,NaN,2014-08-13,13:03:00,13
2804,66432,66432,INCIDENTE GRAVE,VITÓRIA,ES,NaN,2017-02-21,11:47:00,12


In [108]:
#ocorrencias com mais de 10 recomendações, e mostrando somente as cidades e aeródromo
filtro = df.total_recomendacoes > 10
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
844,47938,47938,INCIDENTE,BRASÍLIA,DF,SBBR,2013-04-13,18:00:00,11
1669,52265,52265,ACIDENTE,SANTOS,SP,NaN,2014-08-13,13:03:00,13
2804,66432,66432,INCIDENTE GRAVE,VITÓRIA,ES,NaN,2017-02-21,11:47:00,12


In [109]:
#ocorrencias com classificação de ocorrência == INCIDENTE GRAVE
filtro = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
df.loc[filtro, ['ocorrencia_cidade', 'total_recomendacoes']] #aqui já adicionamos outro filtro, mostrando só duas colunas

,ocorrencia_cidade,total_recomendacoes
6,CAMPINAS,0
12,MARABÁ,0
18,CAMOCIM,0
19,MUANÁ,0
24,RONDONÓPOLIS,0
...,...,...
5139,NOVA BANDEIRANTES,0
5146,FORMOSO DO ARAGUAIA,0
5158,ARAÇATUBA,0
5159,SANTA RITA,0


In [57]:
#ocorrencias com classificação de ocorrência == INCIDENTE GRAVE
filtro = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
6,50713,50713,INCIDENTE GRAVE,CAMPINAS,SP,SDAI,2012-01-07,18:15:00,0
12,45396,45396,INCIDENTE GRAVE,MARABÁ,PA,SBMA,2012-01-11,11:21:00,0
18,50714,50714,INCIDENTE GRAVE,CAMOCIM,CE,SNWC,2012-01-17,18:25:00,0
19,45399,45399,INCIDENTE GRAVE,MUANÁ,PA,NaN,2012-01-18,17:35:00,0
24,45582,45582,INCIDENTE GRAVE,RONDONÓPOLIS,MT,SWRD,2012-01-20,17:00:00,0
...,...,...,...,...,...,...,...,...,...
5139,80427,80427,INCIDENTE GRAVE,NOVA BANDEIRANTES,MT,NaN,2021-12-12,17:20:00,0
5146,80448,80448,INCIDENTE GRAVE,FORMOSO DO ARAGUAIA,TO,NaN,2021-12-18,19:00:00,0
5158,80454,80454,INCIDENTE GRAVE,ARAÇATUBA,SP,SBAU,2021-12-29,21:35:00,0
5159,80455,80455,INCIDENTE GRAVE,SANTA RITA,PB,NaN,2021-12-29,18:50:00,0


In [58]:
#ocorrencias com classificação de ocorrência == INCIDENTE GRAVE e Estado == SP
#criamos duas variaveis para armazenar filtros
filtro1 = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 & filtro2] #mostrar filtro1 e filtro2

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
6,50713,50713,INCIDENTE GRAVE,CAMPINAS,SP,SDAI,2012-01-07,18:15:00,0
78,45598,45598,INCIDENTE GRAVE,BRAGANÇA PAULISTA,SP,SBBP,2012-02-20,14:10:00,0
86,45653,45653,INCIDENTE GRAVE,GUARULHOS,SP,SBGR,2012-02-24,20:04:00,0
91,45599,45599,INCIDENTE GRAVE,BRAGANÇA PAULISTA,SP,SBBP,2012-02-27,19:15:00,0
140,45651,45651,INCIDENTE GRAVE,ITU,SP,NaN,2012-03-24,20:45:00,1
...,...,...,...,...,...,...,...,...,...
4954,80200,80200,INCIDENTE GRAVE,BRAGANÇA PAULISTA,SP,SDVH,2021-07-25,12:25:00,0
4980,80238,80238,INCIDENTE GRAVE,VOTUPORANGA,SP,SDVG,2021-08-11,19:09:00,0
5003,80265,80265,INCIDENTE GRAVE,SÃO PAULO,SP,SBMT,2021-08-31,16:50:00,0
5098,80382,80382,INCIDENTE GRAVE,PIRACICABA,SP,SDPW,2021-11-16,20:50:00,0


In [59]:
#ocorrencias com classificação de ocorrência == INCIDENTE GRAVE e Estado == SP
filtro1 = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 | filtro2] #diferenção filtro1 ou filtro2

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
3,45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,NaN,2012-01-06,17:00:00,0
5,52243,52243,INCIDENTE,UBATUBA,SP,NaN,2012-01-06,14:30:00,0
6,50713,50713,INCIDENTE GRAVE,CAMPINAS,SP,SDAI,2012-01-07,18:15:00,0
12,45396,45396,INCIDENTE GRAVE,MARABÁ,PA,SBMA,2012-01-11,11:21:00,0
...,...,...,...,...,...,...,...,...,...
5158,80454,80454,INCIDENTE GRAVE,ARAÇATUBA,SP,SBAU,2021-12-29,21:35:00,0
5159,80455,80455,INCIDENTE GRAVE,SANTA RITA,PB,NaN,2021-12-29,18:50:00,0
5161,80456,80456,INCIDENTE,SÃO PAULO,SP,SBSP,2021-12-30,13:15:00,0
5163,80452,80452,ACIDENTE,MARACAÍ,SP,NaN,2021-12-31,09:30:00,0


In [60]:
#ocorrencias com classificação de (ocorrência == INCIDENTE GRAVE ou == INCIDENTE) e (Estado == SP)
filtro1 = (df.ocorrencia_classificacao == 'INCIDENTE GRAVE') | (df.ocorrencia_classificacao == 'INCIDENTE')
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
5,52243,52243,INCIDENTE,UBATUBA,SP,NaN,2012-01-06,14:30:00,0
6,50713,50713,INCIDENTE GRAVE,CAMPINAS,SP,SDAI,2012-01-07,18:15:00,0
28,52248,52248,INCIDENTE,SÃO PAULO,SP,SBMT,2012-01-21,11:15:00,0
35,52054,52054,INCIDENTE,SÃO PAULO,SP,SBMT,2012-01-25,23:00:00,0
36,52055,52055,INCIDENTE,RIBEIRÃO PRETO,SP,NaN,2012-01-25,09:30:00,0
...,...,...,...,...,...,...,...,...,...
5134,80425,80425,INCIDENTE,SÃO PAULO,SP,SBSP,2021-12-09,20:30:00,0
5143,80431,80431,INCIDENTE,SANTOS,SP,SBST,2021-12-15,12:30:00,0
5157,80453,80453,INCIDENTE,CAMPINAS,SP,SBKP,2021-12-29,09:00:00,0
5158,80454,80454,INCIDENTE GRAVE,ARAÇATUBA,SP,SBAU,2021-12-29,21:35:00,0


In [61]:
#ocorrencias com classificação de (ocorrência == INCIDENTE GRAVE ou == INCIDENTE) e (Estado == SP)... usando isin()
filtro1 = df.ocorrencia_classificacao.isin(['INCIDENTE GRAVE', 'INCIDENTE'])
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
5,52243,52243,INCIDENTE,UBATUBA,SP,NaN,2012-01-06,14:30:00,0
6,50713,50713,INCIDENTE GRAVE,CAMPINAS,SP,SDAI,2012-01-07,18:15:00,0
28,52248,52248,INCIDENTE,SÃO PAULO,SP,SBMT,2012-01-21,11:15:00,0
35,52054,52054,INCIDENTE,SÃO PAULO,SP,SBMT,2012-01-25,23:00:00,0
36,52055,52055,INCIDENTE,RIBEIRÃO PRETO,SP,NaN,2012-01-25,09:30:00,0
...,...,...,...,...,...,...,...,...,...
5134,80425,80425,INCIDENTE,SÃO PAULO,SP,SBSP,2021-12-09,20:30:00,0
5143,80431,80431,INCIDENTE,SANTOS,SP,SBST,2021-12-15,12:30:00,0
5157,80453,80453,INCIDENTE,CAMPINAS,SP,SBKP,2021-12-29,09:00:00,0
5158,80454,80454,INCIDENTE GRAVE,ARAÇATUBA,SP,SBAU,2021-12-29,21:35:00,0


In [62]:
#ocorrencias cujas cidades começa com a letra C
filtro1 = df.ocorrencia_cidade.str[0] == 'C'
df.loc[filtro1]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
6,50713,50713,INCIDENTE GRAVE,CAMPINAS,SP,SDAI,2012-01-07,18:15:00,0
8,45391,45391,ACIDENTE,CONCEIÇÃO DAS ALAGOAS,MG,NaN,2012-01-08,16:00:00,0
15,45409,45409,ACIDENTE,CÁCERES,MT,NaN,2012-01-14,10:00:00,0
18,50714,50714,INCIDENTE GRAVE,CAMOCIM,CE,SNWC,2012-01-17,18:25:00,0
22,45390,45390,ACIDENTE,CACHOEIRA DOURADA,GO,NaN,2012-01-20,21:00:00,0
...,...,...,...,...,...,...,...,...,...
5142,80434,80434,ACIDENTE,CARACOL,MS,NaN,2021-12-14,21:35:00,0
5144,80447,80447,INCIDENTE,CONFINS,MG,SBCF,2021-12-15,17:10:00,0
5150,80445,80445,ACIDENTE,CHUPINGUAIA,RO,NaN,2021-12-20,12:05:00,0
5157,80453,80453,INCIDENTE,CAMPINAS,SP,SBKP,2021-12-29,09:00:00,0


In [63]:
#ocorrencias cujas cidades terminam com a letra A
filtro1 = df.ocorrencia_cidade.str[-1] == 'A'
df.loc[filtro1]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
5,52243,52243,INCIDENTE,UBATUBA,SP,NaN,2012-01-06,14:30:00,0
9,52244,52244,INCIDENTE,UBERLÂNDIA,MG,SBUL,2012-01-08,22:13:00,0
17,52246,52246,INCIDENTE,BRASÍLIA,DF,SBBR,2012-01-16,16:47:00,0
21,45392,45392,ACIDENTE,BRASÍLIA,DF,NaN,2012-01-19,21:30:00,2
22,45390,45390,ACIDENTE,CACHOEIRA DOURADA,GO,NaN,2012-01-20,21:00:00,0
...,...,...,...,...,...,...,...,...,...
5152,80465,80465,INCIDENTE,VITÓRIA,ES,SBVT,2021-12-21,22:25:00,0
5158,80454,80454,INCIDENTE GRAVE,ARAÇATUBA,SP,SBAU,2021-12-29,21:35:00,0
5159,80455,80455,INCIDENTE GRAVE,SANTA RITA,PB,NaN,2021-12-29,18:50:00,0
5165,80460,80460,INCIDENTE,CURITIBA,PR,SBBI,2021-12-31,15:12:00,0


In [64]:
#ocorrencias cujas cidades terminam com as letras MA
filtro1 = df.ocorrencia_cidade.str[-2:] == 'MA'
df.loc[filtro1]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
249,45939,45939,ACIDENTE,UMUARAMA,PR,NaN,2012-05-30,19:00:00,0
273,46180,46180,ACIDENTE,AURIFLAMA,SP,NaN,2012-06-16,19:30:00,0
971,49113,49113,INCIDENTE GRAVE,NOVA LIMA,MG,NaN,2013-06-15,12:00:00,4
974,48799,48799,ACIDENTE,CANUTAMA,AM,NaN,2013-06-18,21:30:00,0
1518,80059,80059,INCIDENTE,IVINHEMA,MS,NaN,2014-04-27,11:30:00,0
1529,51347,51347,INCIDENTE,DIADEMA,SP,NaN,2014-05-04,16:00:00,0
1729,52340,52340,ACIDENTE,FAMA,MG,NaN,2014-09-20,15:00:00,1
1840,52832,52832,ACIDENTE,BURITAMA,SP,NaN,2014-12-14,23:40:00,0
1871,52979,52979,ACIDENTE,IVINHEMA,MS,NaN,2015-01-02,14:50:00,0
1914,52984,52984,ACIDENTE,NOVA LIMA,MG,SJLY,2015-02-01,15:10:00,0


In [65]:
#ocorrencias cujas cidades contem as letras MA (qualquer parte da palavra)
filtro1 = df.ocorrencia_cidade.str.contains('MA')
df.loc[filtro1]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
12,45396,45396,INCIDENTE GRAVE,MARABÁ,PA,SBMA,2012-01-11,11:21:00,0
16,45693,45693,INCIDENTE,MACAÉ,RJ,NaN,2012-01-14,10:30:00,0
45,45682,45682,INCIDENTE,MACAÉ,RJ,NaN,2012-02-01,13:50:00,0
59,45580,45580,INCIDENTE GRAVE,MACAÉ,RJ,NaN,2012-02-07,14:50:00,4
79,45715,45715,INCIDENTE,MANAUS,AM,SWFN,2012-02-20,09:10:00,0
...,...,...,...,...,...,...,...,...,...
5138,80428,80428,ACIDENTE,SANTA VITÓRIA DO PALMAR,RS,NaN,2021-12-11,19:40:00,0
5153,80461,80461,INCIDENTE,MARINGÁ,PR,SBMG,2021-12-24,03:30:00,0
5156,80463,80463,INCIDENTE,MARINGÁ,PR,SBMG,2021-12-28,16:11:00,0
5160,80451,80451,INCIDENTE,MANAUS,AM,SBEG,2021-12-30,14:41:00,0


In [66]:
#ocorrencias cujas cidades contem as letras MA ou AL (qualquer parte da palavra)
filtro1 = df.ocorrencia_cidade.str.contains('MA|AL')
df.loc[filtro1]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0
8,45391,45391,ACIDENTE,CONCEIÇÃO DAS ALAGOAS,MG,NaN,2012-01-08,16:00:00,0
11,52245,52245,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-09,13:36:00,0
12,45396,45396,INCIDENTE GRAVE,MARABÁ,PA,SBMA,2012-01-11,11:21:00,0
16,45693,45693,INCIDENTE,MACAÉ,RJ,NaN,2012-01-14,10:30:00,0
...,...,...,...,...,...,...,...,...,...
5153,80461,80461,INCIDENTE,MARINGÁ,PR,SBMG,2021-12-24,03:30:00,0
5154,80468,80468,INCIDENTE,SALVADOR,BA,SBSV,2021-12-26,16:56:00,0
5156,80463,80463,INCIDENTE,MARINGÁ,PR,SBMG,2021-12-28,16:11:00,0
5160,80451,80451,INCIDENTE,MANAUS,AM,SBEG,2021-12-30,14:41:00,0


In [67]:
#ocorrencias do ano 2015
filtro1 = df.ocorrencia_dia.dt.year == 2015
df.loc[filtro1]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
1870,52992,52992,INCIDENTE GRAVE,SALVADOR,BA,SBSV,2015-01-01,11:40:00,3
1871,52979,52979,ACIDENTE,IVINHEMA,MS,NaN,2015-01-02,14:50:00,0
1872,53073,53073,INCIDENTE,TEFÉ,AM,SBTF,2015-01-02,16:28:00,0
1873,53074,53074,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-01-02,20:34:00,0
1874,52976,52976,ACIDENTE,TOLEDO,PR,SBTD,2015-01-04,22:04:00,6
...,...,...,...,...,...,...,...,...,...
2336,60632,60632,INCIDENTE GRAVE,ITABERÁ,SP,NaN,2015-12-24,14:00:00,0
2337,60600,60600,INCIDENTE,GUARULHOS,SP,SBGR,2015-12-25,19:00:00,0
2338,60642,60642,INCIDENTE,SÃO FRANCISCO DO SUL,SC,SSSS,2015-12-26,16:00:00,0
2339,60631,60631,ACIDENTE,MAÇAMBARÁ,RS,NaN,2015-12-28,19:00:00,2


In [68]:
#ocorrencias do ano 2015 e mês == 12
filtro1 = df.ocorrencia_dia.dt.year == 2015
filtro2 = df.ocorrencia_dia.dt.month == 12
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
2302,53573,53573,INCIDENTE,GUARULHOS,SP,SBGR,2015-12-01,02:48:00,0
2303,60601,60601,INCIDENTE,PALMAS,TO,SBPJ,2015-12-01,16:05:00,0
2304,53634,53634,INCIDENTE,PALMAS,TO,SBPJ,2015-12-02,17:45:00,0
2305,53636,53636,INCIDENTE,JUNDIAÍ,SP,SBJD,2015-12-02,17:42:00,0
2306,53575,53575,INCIDENTE,CAMPOS DOS GOYTACAZES,RJ,SBFS,2015-12-03,10:50:00,0
2307,60637,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0
2308,53625,53625,ACIDENTE,TRINDADE,GO,NaN,2015-12-06,13:10:00,3
2309,53626,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1
2310,53628,53628,ACIDENTE,AGUAÍ,SP,NaN,2015-12-08,14:30:00,1
2311,53629,53629,INCIDENTE GRAVE,JALES,SP,SDJL,2015-12-08,10:20:00,0


In [69]:
#ocorrencias do ano 2015 e mês == 12 (igual a anterior, mas com um só filtro)
filtro = (df.ocorrencia_dia.dt.year == 2015) & (df.ocorrencia_dia.dt.month == 12)
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
2302,53573,53573,INCIDENTE,GUARULHOS,SP,SBGR,2015-12-01,02:48:00,0
2303,60601,60601,INCIDENTE,PALMAS,TO,SBPJ,2015-12-01,16:05:00,0
2304,53634,53634,INCIDENTE,PALMAS,TO,SBPJ,2015-12-02,17:45:00,0
2305,53636,53636,INCIDENTE,JUNDIAÍ,SP,SBJD,2015-12-02,17:42:00,0
2306,53575,53575,INCIDENTE,CAMPOS DOS GOYTACAZES,RJ,SBFS,2015-12-03,10:50:00,0
2307,60637,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0
2308,53625,53625,ACIDENTE,TRINDADE,GO,NaN,2015-12-06,13:10:00,3
2309,53626,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1
2310,53628,53628,ACIDENTE,AGUAÍ,SP,NaN,2015-12-08,14:30:00,1
2311,53629,53629,INCIDENTE GRAVE,JALES,SP,SDJL,2015-12-08,10:20:00,0


In [70]:
#ocorrencias do ano 2015 ; mês == 12 ; dia == 8
filtro1 = df.ocorrencia_dia.dt.year == 2015
filtro2 = df.ocorrencia_dia.dt.month == 12
filtro3 = df.ocorrencia_dia.dt.day == 8
df.loc[filtro1 & filtro2 & filtro3] #não importa a ordem dos filtros

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
2310,53628,53628,ACIDENTE,AGUAÍ,SP,NaN,2015-12-08,14:30:00,1
2311,53629,53629,INCIDENTE GRAVE,JALES,SP,SDJL,2015-12-08,10:20:00,0
2312,53631,53631,INCIDENTE,CAMPINAS,SP,SBKP,2015-12-08,16:19:00,0
2313,60636,60636,INCIDENTE,CAXIAS DO SUL,RS,SBCX,2015-12-08,13:00:00,0


In [71]:
#ocorrencias do ano 2015 ; mês == 12 ; dia == do 3 a 8
filtro1 = df.ocorrencia_dia.dt.year == 2015
filtro2 = df.ocorrencia_dia.dt.month == 12
filtro3 = (df.ocorrencia_dia.dt.day > 2) & (df.ocorrencia_dia.dt.day < 9)
df.loc[filtro1 & filtro2 & filtro3] #não importa a ordem dos filtros

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
2306,53575,53575,INCIDENTE,CAMPOS DOS GOYTACAZES,RJ,SBFS,2015-12-03,10:50:00,0
2307,60637,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0
2308,53625,53625,ACIDENTE,TRINDADE,GO,NaN,2015-12-06,13:10:00,3
2309,53626,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1
2310,53628,53628,ACIDENTE,AGUAÍ,SP,NaN,2015-12-08,14:30:00,1
2311,53629,53629,INCIDENTE GRAVE,JALES,SP,SDJL,2015-12-08,10:20:00,0
2312,53631,53631,INCIDENTE,CAMPINAS,SP,SBKP,2015-12-08,16:19:00,0
2313,60636,60636,INCIDENTE,CAXIAS DO SUL,RS,SBCX,2015-12-08,13:00:00,0


In [72]:
#para juntar colunas de tipos de dados diferentes numa nova
df['ocorrencia_dia_hora'] = pd.to_datetime(df.ocorrencia_dia.astype(str) + ' ' + df.ocorrencia_hora)

In [73]:
df.head()

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0,2012-01-05 20:27:00
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3,2012-01-06 13:44:00
2,45333,45333,ACIDENTE,VIAMÃO,RS,NaN,2012-01-06,13:00:00,0,2012-01-06 13:00:00
3,45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,NaN,2012-01-06,17:00:00,0,2012-01-06 17:00:00
4,45407,45407,ACIDENTE,SÃO SEPÉ,RS,NaN,2012-01-06,16:30:00,0,2012-01-06 16:30:00


In [74]:
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
ocorrencia_dia_hora         datetime64[ns]
dtype: object

In [75]:
#repetindo o filtro usando a nova coluna criada
#ocorrencias do ano 2015 ; mês == 12 ; dia == do 3 a 8
filtro1 = df.ocorrencia_dia_hora.dt.year == 2015
filtro2 = df.ocorrencia_dia_hora.dt.month == 12
filtro3 = (df.ocorrencia_dia_hora.dt.day > 2) & (df.ocorrencia_dia_hora.dt.day < 9)
df.loc[filtro1 & filtro2 & filtro3] #não importa a ordem dos filtros

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
2306,53575,53575,INCIDENTE,CAMPOS DOS GOYTACAZES,RJ,SBFS,2015-12-03,10:50:00,0,2015-12-03 10:50:00
2307,60637,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0,2015-12-03 16:47:00
2308,53625,53625,ACIDENTE,TRINDADE,GO,NaN,2015-12-06,13:10:00,3,2015-12-06 13:10:00
2309,53626,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1,2015-12-06 15:00:00
2310,53628,53628,ACIDENTE,AGUAÍ,SP,NaN,2015-12-08,14:30:00,1,2015-12-08 14:30:00
2311,53629,53629,INCIDENTE GRAVE,JALES,SP,SDJL,2015-12-08,10:20:00,0,2015-12-08 10:20:00
2312,53631,53631,INCIDENTE,CAMPINAS,SP,SBKP,2015-12-08,16:19:00,0,2015-12-08 16:19:00
2313,60636,60636,INCIDENTE,CAXIAS DO SUL,RS,SBCX,2015-12-08,13:00:00,0,2015-12-08 13:00:00


In [76]:
#ocorrencias do ano 2015 ; mês == 12 ; dia == do 3 a 8; limitando as horas
filtro1 = df.ocorrencia_dia_hora >= '2015-12-03 11:00:00'
filtro2 = df.ocorrencia_dia_hora <= '2015-12-08 13:00:00'
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
2307,60637,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0,2015-12-03 16:47:00
2308,53625,53625,ACIDENTE,TRINDADE,GO,NaN,2015-12-06,13:10:00,3,2015-12-06 13:10:00
2309,53626,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1,2015-12-06 15:00:00
2311,53629,53629,INCIDENTE GRAVE,JALES,SP,SDJL,2015-12-08,10:20:00,0,2015-12-08 10:20:00
2313,60636,60636,INCIDENTE,CAXIAS DO SUL,RS,SBCX,2015-12-08,13:00:00,0,2015-12-08 13:00:00


In [77]:
#ocorrencias do ano 2015 e mês == 3
filtro1 = df.ocorrencia_dia.dt.year == 2015
filtro2 = df.ocorrencia_dia.dt.month == 3
df201503 = df.loc[filtro1 & filtro2] #atribuimos o resultado num novo dataframe
df201503 #impressão

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
1960,53183,53183,ACIDENTE,AMAPORÃ,PR,NaN,2015-03-02,23:00:00,0,2015-03-02 23:00:00
1961,53120,53120,ACIDENTE,CHAVES,PA,SNXW,2015-03-04,13:30:00,0,2015-03-04 13:30:00
1962,53109,53109,ACIDENTE,CAMPO GRANDE,MS,SSIE,2015-03-05,13:50:00,0,2015-03-05 13:50:00
1963,53112,53112,ACIDENTE,MOGI GUAÇU,SP,NaN,2015-03-06,21:00:00,1,2015-03-06 21:00:00
1964,53152,53152,INCIDENTE GRAVE,RIO DE JANEIRO,RJ,SBRJ,2015-03-10,11:30:00,2,2015-03-10 11:30:00
1965,53167,53167,INCIDENTE,MARABÁ,PA,SBMA,2015-03-10,17:33:00,0,2015-03-10 17:33:00
1966,53596,53596,INCIDENTE,ITAPEMA,SC,NaN,2015-03-10,21:35:00,0,2015-03-10 21:35:00
1967,53149,53149,ACIDENTE,TABATINGA,AM,SBTT,2015-03-11,23:25:00,0,2015-03-11 23:25:00
1968,53148,53148,ACIDENTE,ARARAS,SP,SDEH,2015-03-12,12:40:00,0,2015-03-12 12:40:00
1969,53153,53153,INCIDENTE,PORTO ALEGRE,RS,SBPA,2015-03-13,15:45:00,0,2015-03-13 15:45:00


In [78]:
df201503.count()

codigo_ocorrencia           37
codigo_ocorrencia2          37
ocorrencia_classificacao    37
ocorrencia_cidade           37
ocorrencia_uf               37
ocorrencia_aerodromo        21
ocorrencia_dia              37
ocorrencia_hora             37
total_recomendacoes         37
ocorrencia_dia_hora         37
dtype: int64

In [79]:
#agrupamento por coluna "ocorrencia_classificacao" e contar quantas repetições temos por coluna informada
df201503.groupby(['ocorrencia_classificacao']).ocorrencia_classificacao.count()

ocorrencia_classificacao
ACIDENTE           15
INCIDENTE          17
INCIDENTE GRAVE     5
Name: ocorrencia_classificacao, dtype: int64

In [110]:
#agrupamento por coluna "ocorrencia_classificacao" e contar quantas repetições temos por coluna informada
df201503.groupby(['ocorrencia_classificacao']).ocorrencia_aerodromo.count()

ocorrencia_classificacao
ACIDENTE            5
INCIDENTE          14
INCIDENTE GRAVE     2
Name: ocorrencia_aerodromo, dtype: int64

In [80]:
#agrupamento por coluna "ocorrencia_classificacao" e contar quantas repetições temos por código
df201503.groupby(['ocorrencia_classificacao']).size()

ocorrencia_classificacao
ACIDENTE           15
INCIDENTE          17
INCIDENTE GRAVE     5
dtype: int64

In [81]:
#agrupamento por coluna "ocorrencia_classificacao" e contar quantas repetições temos por código, E ORDENADO
df201503.groupby(['ocorrencia_classificacao']).size().sort_values()

ocorrencia_classificacao
INCIDENTE GRAVE     5
ACIDENTE           15
INCIDENTE          17
dtype: int64

In [82]:
#agrupamento por coluna "ocorrencia_classificacao" e contar quantas repetições temos por código, E ORDENADO DECRECENTE
df201503.groupby(['ocorrencia_classificacao']).size().sort_values(ascending=False)

ocorrencia_classificacao
INCIDENTE          17
ACIDENTE           15
INCIDENTE GRAVE     5
dtype: int64

In [83]:
#agrupamento por ocorrencia numa região do pais num ano
#exemplo: região SUDESTE no ano 2015
filtro1 = df.ocorrencia_dia.dt.year == 2015
filtro2 = df.ocorrencia_uf.isin(['SP', 'MG','ES','RJ'])
dfsudeste2015 = df.loc[filtro1 & filtro2]
dfsudeste2015

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
1873,53074,53074,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-01-02,20:34:00,0,2015-01-02 20:34:00
1875,53075,53075,INCIDENTE,JUNDIAÍ,SP,SBJD,2015-01-05,10:19:00,0,2015-01-05 10:19:00
1876,53076,53076,INCIDENTE,BRAGANÇA PAULISTA,SP,NaN,2015-01-06,11:30:00,0,2015-01-06 11:30:00
1879,53072,53072,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-01-08,21:13:00,0,2015-01-08 21:13:00
1880,52994,52994,INCIDENTE,SÃO PAULO,SP,SBMT,2015-01-09,23:25:00,0,2015-01-09 23:25:00
...,...,...,...,...,...,...,...,...,...,...
2332,60602,60602,INCIDENTE,RIBEIRÃO PRETO,SP,SBRP,2015-12-22,06:27:00,0,2015-12-22 06:27:00
2333,60615,60615,INCIDENTE,SÃO PAULO,SP,SBMT,2015-12-22,13:07:00,0,2015-12-22 13:07:00
2335,60641,60641,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-23,15:53:00,0,2015-12-23 15:53:00
2336,60632,60632,INCIDENTE GRAVE,ITABERÁ,SP,NaN,2015-12-24,14:00:00,0,2015-12-24 14:00:00


In [84]:
#Tipos de ocorrencia na região sudeste em 2015
dfsudeste2015.groupby('ocorrencia_classificacao').size()

ocorrencia_classificacao
ACIDENTE            60
INCIDENTE          128
INCIDENTE GRAVE     19
dtype: int64

In [112]:
dfsudeste2015.count() #a soma dos valores anteriores confere:207

codigo_ocorrencia           207
codigo_ocorrencia2          207
ocorrencia_classificacao    207
ocorrencia_cidade           207
ocorrencia_uf               207
ocorrencia_aerodromo        152
ocorrencia_dia              207
ocorrencia_hora             207
total_recomendacoes         207
ocorrencia_dia_hora         207
dtype: int64

In [85]:
#Agrupado por tipos de ocorrencia na região sudeste em 2015; identificando os estados em cada um
dfsudeste2015.groupby(['ocorrencia_classificacao', 'ocorrencia_uf']).size()

ocorrencia_classificacao  ocorrencia_uf
ACIDENTE                  ES                1
                          MG               18
                          RJ                6
                          SP               35
INCIDENTE                 ES                2
                          MG               31
                          RJ               17
                          SP               78
INCIDENTE GRAVE           ES                1
                          MG                2
                          RJ                7
                          SP                9
dtype: int64

In [86]:
#Agrupando por estados (UF) e identificando os tipos de ocorrencia em cada um
dfsudeste2015.groupby(['ocorrencia_uf','ocorrencia_classificacao']).size()

ocorrencia_uf  ocorrencia_classificacao
ES             ACIDENTE                     1
               INCIDENTE                    2
               INCIDENTE GRAVE              1
MG             ACIDENTE                    18
               INCIDENTE                   31
               INCIDENTE GRAVE              2
RJ             ACIDENTE                     6
               INCIDENTE                   17
               INCIDENTE GRAVE              7
SP             ACIDENTE                    35
               INCIDENTE                   78
               INCIDENTE GRAVE              9
dtype: int64

In [87]:
#Tipos de ocorrencia na região sudeste em 2015; por cidades
dfsudeste2015.groupby(['ocorrencia_cidade']).size().sort_values(ascending=False)

ocorrencia_cidade
SÃO PAULO          25
RIO DE JANEIRO     19
BELO HORIZONTE     18
JUNDIAÍ            14
GUARULHOS          10
                   ..
ITARIRI             1
ITAPIRA             1
ITABERÁ             1
IBATÉ               1
ÁLVARES MACHADO     1
Length: 86, dtype: int64

In [88]:
#Analisando uma cidade específica...
filtro = dfsudeste2015.ocorrencia_cidade == 'RIO DE JANEIRO'
dfsudeste2015.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
1929,53001,53001,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2015-02-08,19:07:00,0,2015-02-08 19:07:00
1932,52995,52995,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2015-02-09,10:00:00,0,2015-02-09 10:00:00
1941,53132,53132,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2015-02-16,12:39:00,0,2015-02-16 12:39:00
1945,53125,53125,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2015-02-19,15:53:00,0,2015-02-19 15:53:00
1955,53118,53118,ACIDENTE,RIO DE JANEIRO,RJ,NaN,2015-02-26,12:30:00,0,2015-02-26 12:30:00
1964,53152,53152,INCIDENTE GRAVE,RIO DE JANEIRO,RJ,SBRJ,2015-03-10,11:30:00,2,2015-03-10 11:30:00
1979,53188,53188,INCIDENTE,RIO DE JANEIRO,RJ,SBJR,2015-03-20,21:00:00,0,2015-03-20 21:00:00
2020,53234,53234,ACIDENTE,RIO DE JANEIRO,RJ,NaN,2015-04-10,12:10:00,0,2015-04-10 12:10:00
2029,53235,53235,ACIDENTE,RIO DE JANEIRO,RJ,NaN,2015-04-21,13:00:00,0,2015-04-21 13:00:00
2175,53453,53453,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2015-08-19,13:00:00,0,2015-08-19 13:00:00


In [89]:
#Analisando uma cidade específica... e somamos o 'total de recomendacoes'
filtro = dfsudeste2015.ocorrencia_cidade == 'RIO DE JANEIRO'
dfsudeste2015.loc[filtro].total_recomendacoes.sum()

6

In [90]:
#Analisando uma cidade específica... e filtro por 'total de recomendacoes' que não sejam nulos
filtro1 = dfsudeste2015.ocorrencia_cidade == 'RIO DE JANEIRO'
filtro2 = dfsudeste2015.total_recomendacoes > 0
dfsudeste2015.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
1964,53152,53152,INCIDENTE GRAVE,RIO DE JANEIRO,RJ,SBRJ,2015-03-10,11:30:00,2,2015-03-10 11:30:00
2326,60620,60620,ACIDENTE,RIO DE JANEIRO,RJ,SBJR,2015-12-15,16:50:00,4,2015-12-15 16:50:00


In [91]:
#Total de recomendações por cidade na região sudeste
dfsudeste2015.groupby(['ocorrencia_cidade']).total_recomendacoes.sum().sort_values(ascending=False)

ocorrencia_cidade
RIO DE JANEIRO     6
CARAPICUÍBA        5
BELO HORIZONTE     4
PIRACICABA         2
OURO PRETO         2
                  ..
IBATÉ              0
GUARULHOS          0
GUARDA-MOR         0
GUARAPARI          0
ÁLVARES MACHADO    0
Name: total_recomendacoes, Length: 86, dtype: int64

In [113]:
#Total de recomendações por aerodromo.. na região sudeste (dropna = True para eliminar os dados não informados (NaN))
dfsudeste2015.groupby(['ocorrencia_aerodromo'], dropna=True).total_recomendacoes.sum().sort_values(ascending=False)

ocorrencia_aerodromo
SBPR    4
SBJR    4
SBRJ    2
SBME    2
SDPW    2
SNVC    1
SDET    1
SDAI    1
SBBP    1
SDIO    1
SBGW    1
SDRK    0
SBBH    0
SDMH    0
SDJO    0
SDJC    0
SDJL    0
SDVE    0
SDTF    0
SDEH    0
SDVH    0
SIBH    0
SIVU    0
SJLY    0
SNBR    0
SNDV    0
SNHB    0
SNKF    0
SNPA    0
SDEM    0
SBVT    0
SDCO    0
SDAM    0
SBBU    0
SBCB    0
SBCF    0
SBFS    0
SBGL    0
SBGR    0
SBGV    0
SBJD    0
SBKP    0
SBML    0
SBMT    0
SBPC    0
SBRP    0
SBSJ    0
SBSP    0
SBSR    0
SBST    0
SBUL    0
SBUR    0
SSOL    0
Name: total_recomendacoes, dtype: int64

In [93]:
#Só quero as cidades que tem recomendações _ tirar os '0' (usamos filtro para >0)
filtro = dfsudeste2015.total_recomendacoes > 0
dfsudeste2015.loc[filtro].groupby(['ocorrencia_cidade']).total_recomendacoes.sum().sort_values(ascending=False)

ocorrencia_cidade
RIO DE JANEIRO       6
CARAPICUÍBA          5
BELO HORIZONTE       4
MACAÉ                2
PIRACICABA           2
OURO PRETO           2
ITARIRI              2
AGUAÍ                1
TIETÊ                1
MOGI GUAÇU           1
MIRACATU             1
ITÁPOLIS             1
JUNDIAÍ              1
AMERICANA            1
ITAPIRA              1
GUARATINGUETÁ        1
CATANDUVA            1
CABECEIRA GRANDE     1
BRAGANÇA PAULISTA    1
VIÇOSA               1
Name: total_recomendacoes, dtype: int64

In [94]:
#Só quero as cidades que tem recomendações _ tirar os '0' (usamos filtro) + separar por mês
filtro = dfsudeste2015.total_recomendacoes > 0
dfsudeste2015.loc[filtro].groupby(['ocorrencia_cidade', dfsudeste2015.ocorrencia_dia.dt.month]).total_recomendacoes.sum()

ocorrencia_cidade  ocorrencia_dia
AGUAÍ              12                1
AMERICANA          12                1
BELO HORIZONTE     8                 4
BRAGANÇA PAULISTA  11                1
CABECEIRA GRANDE   5                 1
CARAPICUÍBA        4                 5
CATANDUVA          4                 1
GUARATINGUETÁ      6                 1
ITAPIRA            2                 1
ITARIRI            8                 2
ITÁPOLIS           12                1
JUNDIAÍ            11                1
MACAÉ              6                 2
MIRACATU           11                1
MOGI GUAÇU         3                 1
OURO PRETO         6                 2
PIRACICABA         6                 2
RIO DE JANEIRO     3                 2
                   12                4
TIETÊ              5                 1
VIÇOSA             5                 1
Name: total_recomendacoes, dtype: int64

In [116]:
#cidade filtrada por recomendações >0
filtro1 = dfsudeste2015.total_recomendacoes > 0
filtro2 = dfsudeste2015.ocorrencia_cidade == 'RIO DE JANEIRO'
dfsudeste2015.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
1964,53152,53152,INCIDENTE GRAVE,RIO DE JANEIRO,RJ,SBRJ,2015-03-10,11:30:00,2,2015-03-10 11:30:00
2326,60620,60620,ACIDENTE,RIO DE JANEIRO,RJ,SBJR,2015-12-15,16:50:00,4,2015-12-15 16:50:00
